In [1]:
#推荐系统-Average_Filling实现

In [2]:
import pandas as pd
import numpy as np

In [3]:
uname = ["user_id", "item_id", "rating", "timestamp"]
u1base = pd.read_csv("../TestData/ml-100k/u1.base", delimiter = '\t', header = None, names = uname)
u1base.head()

,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [4]:
u1base.describe()

,user_id,item_id,rating,timestamp
count,80000.00000,80000.000000,80000.000000,8.000000e+04
mean,525.16570,425.705200,3.528350,8.835645e+08
std,255.94956,331.383936,1.118565,5.318611e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,334.00000,175.000000,3.000000,8.794556e+08
50%,535.00000,321.000000,4.000000,8.828441e+08
75%,744.00000,631.000000,4.000000,8.882676e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [5]:
#活动user数量和item的数量
n = u1base.user_id.max()
m = u1base.item_id.max()
print(["n", "m"])
print([n, m])

['n', 'm']
[943, 1682]


In [6]:
#检查每个user，每个item是否都有数据
#item集合元素数量小于最大id：说明存在某些item没有数据，后续操作需要特殊处理
[len(set(list(u1base.user_id))), len(set(list(u1base.item_id)))]

[943, 1650]

In [7]:
ru = np.array([u1base.loc[u1base.user_id == x + 1].rating.mean() for x in range(n)])
ri = np.array([u1base.loc[u1base.item_id == x + 1].rating.mean() for x in range(m)])
rui = u1base.rating.mean()
np.bincount(np.isnan(ri))

array([1650,   32])

In [8]:
#将缺少数据的item的分数平均值用全局平均值代替
for u in range(m):
    if np.isnan(ri[u]):
        ri[u] = rui
np.bincount(np.isnan(ri))

array([1682])

In [9]:
bu = [np.mean(u1base.loc[u1base.user_id == x + 1].rating - ri[u1base.loc[u1base.user_id == x + 1].item_id - 1]) for x in range(n)]
bi = [np.mean(u1base.loc[u1base.item_id == x + 1].rating - ru[u1base.loc[u1base.item_id == x + 1].user_id - 1]) for x in range(m)]
np.bincount(np.isnan(bi))

array([1650,   32])

In [10]:
#将缺少数据的item的bias用0代替
for u in range(m):
    if np.isnan(bi[u]):
        bi[u] = 0
np.bincount(np.isnan(bi))

array([1682])

In [11]:
u1test = pd.read_csv("../TestData/ml-100k/u1.test", delimiter = '\t', header = None, names = uname)
u1test

,user_id,item_id,rating,timestamp
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198
...,...,...,...,...
19995,458,648,4,886395899
19996,458,1101,4,886397931
19997,459,934,3,879563639
19998,460,10,3,882912371


In [12]:
testn = len(u1test)

In [13]:
pred0 = np.array([ru[u1test.loc[x].user_id - 1] for x in range(testn)])
pred1 = np.array([ri[u1test.loc[x].item_id - 1] for x in range(testn)])
pred2 = (pred0 + pred1) / 2
pred3 = np.array([bu[u1test.loc[x].user_id - 1] for x in range(testn)]) + pred1
pred4 = np.array([bi[u1test.loc[x].item_id - 1] for x in range(testn)]) + pred0
pred5 = rui + np.array([bu[u1test.loc[x].user_id - 1] for x in range(testn)]) + np.array([bi[u1test.loc[x].item_id - 1] for x in range(testn)])

In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [15]:
pred = {
    "user_average": [mean_absolute_error(pred0, u1test.rating), mean_squared_error(pred0, u1test.rating) ** 0.5],
    "item_average": [mean_absolute_error(pred1, u1test.rating), mean_squared_error(pred1, u1test.rating) ** 0.5],
    "maen_of_user_average_and_item_average": [mean_absolute_error(pred2, u1test.rating), mean_squared_error(pred2, u1test.rating) ** 0.5],
    "user_bias_and_item_average": [mean_absolute_error(pred3, u1test.rating), mean_squared_error(pred3, u1test.rating) ** 0.5],
    "user_average_and_item_bias": [mean_absolute_error(pred4, u1test.rating), mean_squared_error(pred4, u1test.rating) ** 0.5],
    "global_average_user_bias_and_item_bias": [mean_absolute_error(pred5, u1test.rating), mean_squared_error(pred5, u1test.rating) ** 0.5]
}

In [16]:
final_res = pd.DataFrame(pred, index = ["MAE", "RMSE"]).T

In [17]:
final_res

,MAE,RMSE
user_average,0.850191,1.062995
item_average,0.827568,1.033411
maen_of_user_average_and_item_average,0.808481,0.998513
user_bias_and_item_average,0.757384,0.960243
user_average_and_item_bias,0.769637,0.975786
global_average_user_bias_and_item_bias,0.761279,0.962332
